# Nanocourse - Genomic analysis using sketching techniques




## Sketching

A *data sketch* of data $X$ is the output of a (randomized) function $f$ s.t.:
 - $|f(X)| \subseteq o(|X|)$
 - $f$ perserves some properties of $X$ e.g. approximation of the number of distinct elements
 - $f$ preserves certain similarity measures e.g. number of shared elements
 - $f(X)$ allows to be updated efficiently


## Counting distinct elements of a set

In an **online** setting (stream)


### Naive solution

Using a Hashmap or a Bitvector

**TODO:** Implement naive_counting() in naive_couting.cpp

Test your implementation by running
```
./build/source/naivecounting data/ecoli1_k31_ust.fa.gz
```
The output should be
```
Distinct kmers: 4877400
```
How much memory consumption do you expect?

Let us check that in practice:

In [ ]:
import os
from evaluate.track import track_memory_and_runtime

file = "data/ecoli1_k31_ust.fa.gz"
file_size = os.path.getsize(file)/(1024*1024)
_,memory,_ = track_memory_and_runtime("build/source/naivecounting", file)
print("File size: %.1fMB" % file_size)
print("Memory consumption: %.1fMB" % memory)

#### Observation:
- space consumption linear w.r.t. number distinct elements
- impractical for big data

-> approximate/probabilistic counting

### Flajolet-Martin’s algorithm

#### Recall:

Let $\mathcal{M}$ be a multiset of uniformly distributed random numbers.
 - The cardinality of $\mathcal{M}$ can be estimated by the maximum number of leading zeros in the binary representation of each number in $\mathcal{M}$.
 - If max leading zeros is $l$, one exepcts $2^l$ distinct elements
(the probability of observing a binary encoded number beginning with $k$ zeroes followed by a one is $1/2^{(k+1)}$ ).

#### Algorithm:

- Map each element $x$ to hash $h(x)$
- remember the maximum number $l$ of leading 0-bits seen in any $h(x)$
- estimate cardinality by $2^l$

<!-- % $\Psi \approx 0.77351$ is a normalization constant. -->

**TODO**
- Implement flajolet_martin() in flajolet_martin.cpp
- Test it by running
```
./build/source/flajoletmartin data/ecoli1_k31_ust.fa.gz
```
Let us compare run time, space consumption, and accuracy to exact algorithm:

In [ ]:
import numpy as np
from evaluate.plots import plot_performance, plot_accuracy_FM, plot_accuracies

files = ["data/ecoli1_k31_ust.fa.gz", "data/ecoli2_k31_ust.fa.gz",
         "data/yeast.k31.unitigs.fa.ust.fa.gz", "data/salmonella_100_k31_ust.fa.gz"]

naive_results, fm_results = [], []
for file in files:
    naive_results.append(track_memory_and_runtime("build/source/naivecounting", file))
    fm_results.append(track_memory_and_runtime("build/source/flajoletmartin", file))

file_sizes = [os.path.getsize(file) for file in files]
results = np.array([naive_results, fm_results])
algorithms = ["Naive", "Flajolet Martin"]

plot_performance(results, algorithms, file_sizes)
# plot_accuracy_FM(results, algorithms)
errors=np.abs(results[1,:,2] - results[0,:,2])/results[0,:,2]
print("Error: %.1f+-%.1f%%" % (np.mean(errors)*100, np.std(errors)*100))

#### Observations:
- Large variance! How could you compensate that?
- Less space consumption: hash $h(x) \rightarrow [0,L]$ requires $\log(L) \approx \log(n)$ space for $n$ distinct elements i.e. exponential less space w.r.t. sequence length $n$


### HyperLogLog

#### Refinement:
- split $\mathcal{M}$ into $m$ subsets
- estimate cardinalities of subsets
- return mean

The normalized version of the harmonic mean is the estimate

$$E:=\frac{\alpha_m m^2}{\sum_{j=1}^m 2^{-M(j)}}.$$

for $m$ subsets $M(i)$ and normalization constant $\alpha_m \approx 0.7$.

**TODO:** Implement hyperloglog() in hyperloglog.cpp

Let us compare run time, space consumption, and accuracy to Flajolet-Martin and the exact algorithm:

In [ ]:
naive_results, fm_results, hll_results = [], [], []
for file in files:
    naive_results.append(track_memory_and_runtime("build/source/naivecounting", file))
    fm_results.append(track_memory_and_runtime("build/source/flajoletmartin", file))
    hll_results.append(track_memory_and_runtime("build/source/hyperloglog", file))

file_sizes = [os.path.getsize(file) for file in files]
results = np.array([naive_results, fm_results, hll_results])
algorithms = ["Naive", "Flajolet Martin", "HLL"]

plot_performance(results, algorithms, file_sizes)
plot_accuracies(results, algorithms)
error_fm=np.abs(results[1,:,2] - results[0,:,2])/results[0,:,2]
error_hll=np.abs(results[2,:,2] - results[0,:,2])/results[0,:,2]
print("Total Runtime Hashtable: %.1fs, FM: %.1fs, HLL: %.1fs" % (np.sum(results[0,:,0]), np.sum(results[1,:,0]), np.sum(results[2,:,0])))
print("Maximum Memory Hashtable: %.1fMb, FM: %.1fMb, HLL: %.1fMb" % (np.max(results[0,:,1]), np.max(results[1,:,1]), np.max(results[2,:,1])))
print("Error Flajolet Martin: %.1f+-%.1f%%" % (np.mean(error_fm)*100, np.std(error_fm)*100))
print("Error HyperLogLog: %.1f+-%.1f%%" % (np.mean(error_hll)*100, np.std(error_hll)*100))

#### Observations:

- Smaller variance
- Less space consumption: $O(\log \log n)$


## Take Aways

- Certain tasks for massive data as in molecular biology require data sketching.
- With a bit of randomness, measures for distinct elements in a set become tractable for big data.